In [ ]:
# 라이브러리 import
import pandas as pd
import numpy as np

from selenium import webdriver
from bs4 import BeautifulSoup 
from tqdm.notebook import tqdm
import time
import re

# 카테고리 담기

In [ ]:
browser = webdriver.Chrome('C:\chrome\chromedriver.exe')
url = "https://www.idus.com/c/category/3" # 공예
browser.get(url)
time.sleep(3)

html = browser.page_source
soup=BeautifulSoup(html, 'html.parser')

In [ ]:
category_lists = soup.select('div.inner-w.mobile-full ul.side_nav__sublist li.side_nav__submenu a')

for i in range(0,12):
    print(category_lists[i].text)

cat_data = [] # 빈집 만들기 #공예 해당하는 것
for i in range(0,12):
    cat_list = category_lists[i].text
    cat_num = i+15
    cat_data.append([cat_list, cat_num])  
    
columns = ['카테고리','category_num'] 
cat_df = pd.DataFrame(cat_data, columns=columns)
cat_df

35
캔들, 디퓨저, 방향제
실크스크린, 염색
발향, 룸스프레이
자수, 재봉, 펠트
뜨개, 마크라메
가죽
금속, 악세사리
네온사인, 와이어
유리
목공
도자기
기타


# url 사용자 함수 만들기

In [ ]:
import time
import pandas as pd
import re

def collect_url(page):
    start = time.time()
    url = f'https://www.idus.com/c/category/3/{page}'
    #  url 접속
    browser = webdriver.Chrome('C:\chrome\chromedriver.exe')
    browser.get(url)
    time.sleep(2)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
       
    # 현재 문서 높이를 가져와서 저장
    prev_height = browser.execute_script("return document.body.scrollHeight")

    while True:
        # 스크롤을 가장 아래로 내림
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = browser.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height

        
    #url들 수집
    url_data =[]
    page = f'{page}'

    # URL_raw 크롤링 시작
    articles = ".ui_card__title" # 중간에 띄어쓰기는 .으로 대체
    article_raw = browser.find_elements_by_css_selector(articles)
    article_raw[0]\

    # 1초 쉬어 주고,
    time.sleep(1)

    # 크롤링한 url 정제 시작
    for article in article_raw:
        url = article.get_attribute('href')  
        title = article.text
        title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", title)
        url_data.append([page, url, title])

    # 데이터 프레임 만들기
    columns = ['category_num', 'url', 'title']
    df = pd.DataFrame(url_data, columns = columns)

    # 데이터 프레임 저장하기   
    df_name = './re_url/idus_{}.xlsx'.format(page) # url 폴더 생성하거나 지워서 run
    df.to_excel(df_name, index=False) #,engine='openpyxl'
    
    end = time.time() - start
    print("used time: %dm%ds" % divmod(end,60))
    return(df)

    

In [ ]:
for i in range(15,27):
    collect_url(i)    

In [ ]:
# url 합치기
url_df = pd.DataFrame([])


for i in range(15,27):
    folder = './idus/'
    f_list = 'idus_{}.xlsx'.format(i)
    f_path = folder+f_list
    temp = pd.read_excel(f_path)
    url_df = url_df.append(temp)  
url_df.info()

In [ ]:
url_df.to_excel('./idus/url_all.xlsx', index=False)
url_df

,category_num,url,title
0,15,https://www.idus.com/c/class/4068,향기로운 플라워타블렛과 섬유향수만들기
1,15,https://www.idus.com/c/class/3350,"반짝반짝, 크리스탈캔들 패브릭미스트 클래스"
2,15,https://www.idus.com/c/class/964,경복궁역 향기 가득 플라워 왁스 타블렛 만들기
3,15,https://www.idus.com/c/class/4552,달콤 디저트 까눌레 캔들 톰과제리 치즈캔들 만들기
4,15,https://www.idus.com/c/class/3894,건대입구 젤캔들 홀더와 티라이트4개 원데이클래스
...,...,...,...
449,26,https://www.idus.com/c/class/1998,부산 라탄원목트레이
450,26,https://www.idus.com/c/class/1896,힐링라탄 감성소품 미니스탠드 원데이클래스
451,26,https://www.idus.com/c/class/1688,힐링라탄 타원트레이
452,26,https://www.idus.com/c/class/729,레진머리핀 3개 내 맘에 쏘 옥 드는 레진 머리핀을 만들자


# 댓글 크롤링 코드

In [ ]:
import time
import re
import pandas as pd
from tqdm.notebook import tqdm 

# url 불러오기
url_load = pd.read_excel("./idus/url_all.xlsx",engine='openpyxl')
num_list = len(url_load)
print(num_list) # 총 url 갯수
url_load

start = time.time() 

# number = len(url_load)  # 수집할 글 갯수
# number = num_list

title_list = []
nickname_list = []
datetime_list = []
like_list = []
content_list = []

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, 454)): # 끊긴 경우 여기 0대신해 시작 숫자만 변경하면 됨
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')
    driver.get(url_load.url[i])
    time.sleep(2)
    
    try:
        
        target_info = {} 
        
        try:
             # '참여후기' 클릭
            driver.find_element_by_link_text("참여후기").click( )
            time.sleep(2)
        
            # n: 반복할 횟수 설정, '더보기' 클릭
            n = 4
            i = 0
            while i < n: # 이 조건이 만족되는 동안 반복 실행
                try:
                    time.sleep(2)
                    driver.find_element_by_css_selector("#edu-review > div.pagination > div > button > span").click( ) #더보기
                    i = i+1
                except:
                    break
                    
        except:
            pass

        comments = driver.find_elements_by_css_selector('ul.list-style-review>li')
        for comment in comments:                     
            title = driver.find_element_by_css_selector("h2.sticky_aside__title").text        
            title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", "", title)
            title_list.append(title)

            nickname = comment.find_elements_by_css_selector('a > div.clf > div > div.txt-block > em')[0].text
            nickname = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", "", nickname)
            nickname_list.append(nickname)

            datetime = comment.find_elements_by_css_selector('a > div.clf > div > div.txt-block > time')[0].text
            datetime_list.append(datetime)

            like = comment.find_elements_by_css_selector('a > div.clf > span')[0].get_attribute('data-value')
            like_list.append(like)

            content = comment.find_elements_by_css_selector('a > div.area-txt > div > span')[0].text
            content = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", "", content)
            content_list.append(content)      
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except Exception as e:
        driver.close()
        print('실패', e, type(e))
        continue
        
    
    # 중간,중간에 파일로 저장하기
    if i == 100 or 200 or 300 or 400 :
        columns = ['title', 'nickname', 'datetime', 'like', 'content']
        result_df = pd.DataFrame(target_info,columns=columns)
        result_df.to_excel('./idus/result_0520.xlsx',encoding='utf-8-sig', index=False)
        time.sleep(3)
        

end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))
target_info['title'] = title_list
target_info['nickname'] = nickname_list
target_info['datetime'] = datetime_list
target_info['like'] = like_list
target_info['content'] = content_list


454


  0%|          | 0/454 [00:00<?, ?it/s]

향기로운 플라워타블렛과 섬유향수만들기
반짝반짝 크리스탈캔들패브릭미스트 클래스
경복궁역향기 가득 플라워 왁스 타블렛 만들기
달콤 디저트 까눌레 캔들톰과제리 치즈캔들 만들기
건대입구 젤캔들 홀더와 티라이트4개 원데이클래스
건대입구 플라워 디퓨저  왁스타블렛 세트 원데이클래스
젤 캔들 홀더  티라이트 캔들 4구
왁스타블렛 2구 만들기 티라이트 캔들 증정
시간협의필수 보석은 역시 원석 향기로운 젬스톤 원석캔들 만들기
원목의 감성을 느낄 수 있는 소이왁스 방향제 만들기
원목과 향기 드라이플라워가 만난 하나뿐인 방향제
홍대 아름다운 향기오브제 왁스타블렛과 롤온향수 클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
테라조캔들테라조왁스타블렛 원데이클래스
귀여운 케이크캔들 디저트캔들 오브제캔들 원데이로 만들어요
귀여운 케이크캔들 디저트캔들 오브제캔들 원데이로 만들어요
귀여운 케이크캔들 디저트캔들 오브제캔들 원데이로 만들어요
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
아루온 디저트오브제 캔들 원데이클래스
나만의 탄생화 향수 만들기
감성오브제

유리공예스테인드글라스로 나만의 소품만들기
홍대연남동 유리 전사지원데이클래스
홍대연남동 유리 전사지원데이클래스
집 안을 예쁘게 꾸밀 시간 스테인드글라스 거울입체 만들기
집 안을 예쁘게 꾸밀 시간 스테인드글라스 거울입체 만들기
나만의 예쁜 썬캐쳐만들기 스테인드글라스
유리공예 유리 플레이트에 나만의 감성을 담아보아요
나만의 빛나는 인테리어 소품 만들기스테인드글라스
내손으로 만드는 원목도마원데이클래스
내손으로 만드는 원목도마원데이클래스
내손으로 만드는 원목도마원데이클래스
성수동데이트나만의 도마만들기 도마원데이클래스
수원 탑동 나무공방 원목도마 만들기 수원원데이 클래스
나만의 도마도 만들고 커피한잔의 여유도 즐기고
망원동 세상에 단 하나 접시 만들기
도예원데이 캐릭터 또는 자유플레이트 도자기접시만들기
서울 도자기 맛집무제한 만들기 물레로 코로나를 돌려 없애버리자
원데이 도자기핸드페인팅예쁜 도자기로 홈카페 완성하기
홍대연남동 도자기 전사지원데이클래스
나만의 알록달록 패턴마블링 그릇 1개 만들어요마블링 컵 가능
핸드빌딩 나만의 도자기 소주잔 세트 만들기
마음을 담은 특별한 그릇 만들기
복잡한 서울 속 힐링 물레 체험
을지로 감성도자기 만들기 컵소서 클래스
형태 크기 개수 제한 없는 도자기원데이클래스도자기공방 디세
형태 크기 개수 제한 없는 도자기원데이클래스도자기공방 디세
개그림개성따라 그린 세상에 하나뿐인 도자기 그림
잠실송리단길도자기에 나만의 빈티지감성 드로잉하기
도예판 접시와 물레로 컵 만들기
알록달록 접시 만들기
도자기원데이클래스1인두가지작품물레체험핸드빌딩
알록달록 도자기 패턴마블링 그릇 2개만들기유광무광
알록달록 도자기 패턴마블링 그릇 2개만들기유광무광
알록달록 도자기 패턴마블링 그릇 2개만들기유광무광
알록달록 도자기 패턴마블링 그릇 2개만들기유광무광
알록달록 도자기 패턴마블링 그릇 2개만들기유광무광
내 취향을 가득 담은 도자기만들기
내 취향을 가득 담은 도자기만들기
내 취향을 가득 담은 도자기만들기
내 취향을 가득 담은 도자기만들기
내 취향을 가득 담은 도자기만들

# 엑셀 저장

In [ ]:
import pandas as pd
columns = ['title', 'nickname', 'datetime', 'like', 'content']
result_df = pd.DataFrame(target_info,columns=columns)
result_df.to_excel('./idus/result.xlsx',encoding='utf-8-sig', index=False)
result_df

,title,nickname,datetime,like,content
0,향기로운 플라워타블렛과 섬유향수만들기,빵햄,2021년 5월 11일,5,작년에 지인과 같이 3번의 수업을하고 올해는 혼자 처음으로 받았는데 한결같이 친절하...
1,향기로운 플라워타블렛과 섬유향수만들기,지수,2021년 4월 24일,5,작가님이 정말 친절하게 알려주셨어요 혼자 갔는데 너무 편안하게 대해주셔서 재미있었습...
2,향기로운 플라워타블렛과 섬유향수만들기,윤현정,2021년 4월 7일,5,작가님이 친절하게 잘 안내해 주셔서 향기롭고 행복한 시간이 된 것 같습니다 작품도 ...
3,향기로운 플라워타블렛과 섬유향수만들기,김민지,2021년 3월 27일,5,"직접 원하는 디자인과 향을 골라서 만들수있고, 타블렛이랑 섬유향수 모두 만들 수 있..."
4,향기로운 플라워타블렛과 섬유향수만들기,이은정,2021년 3월 25일,5,"선생님께서 정말 친절하셨구, 자유도가 높은 수업이다보니 만족도도 높은 것 같아요원하..."
...,...,...,...,...,...
884,레진머리핀 3개 내 맘에 쏘옥 드는 레진 머리핀을 만들자,Sunny Blue,2020년 1월 7일,5,그립톡은 만들기는 처음 해봤는데 친절하게 설명해주시고 재료도 다양해서 결과도 잘 나...
885,공간을 그리다 나만의 작품 만들기 와이어아트,야뭄,2021년 3월 3일,5,재밌고 좋았어요
886,공간을 그리다 나만의 작품 만들기 와이어아트,영원,2020년 4월 15일,5,작가님 덕분에 소중한 친구에게 사랑하는 마음 가득 담아 선물할 수 있었어요친절하게 ...
887,공간을 그리다 나만의 작품 만들기 와이어아트,김애린,2019년 12월 8일,5,친한 언니와 색다른 클래스 들어보고 싶어서 많이 알아보다가 딱 마음 맞은 유일한 클...
